# Exploring the data

In [ ]:
#Get the log files

In [1]:
flumeFiles = ("/loudacre/weblogs/FlumeData.*")

In [2]:
accountFiles = ("/loudacre/accounts/part-m*")

In [3]:
baseStation = ("/loudacre/base_stations.tsv")

In [ ]:
# Setup RDS

In [4]:
accounts = sc.textFile(accountFiles).map(lambda line: line.split(','))

In [5]:
basestations = sc.textFile(baseStation).map(lambda line: line.split('\t'))

In [6]:
accounts.take(5)

[[u'1',
  u'2008-10-23 16:05:05.0',
  u'\\N',
  u'Donald',
  u'Becton',
  u'2275 Washburn Street',
  u'Oakland',
  u'CA',
  u'94660',
  u'5100032418',
  u'2014-03-18 13:29:47.0',
  u'2014-03-18 13:29:47.0'],
 [u'2',
  u'2008-11-12 03:00:01.0',
  u'\\N',
  u'Donna',
  u'Jones',
  u'3885 Elliott Street',
  u'San Francisco',
  u'CA',
  u'94171',
  u'4150835799',
  u'2014-03-18 13:29:47.0',
  u'2014-03-18 13:29:47.0'],
 [u'3',
  u'2008-12-21 09:19:50.0',
  u'\\N',
  u'Dorthy',
  u'Chalmers',
  u'4073 Whaley Lane',
  u'San Mateo',
  u'CA',
  u'94479',
  u'6506877757',
  u'2014-03-18 13:29:47.0',
  u'2014-03-18 13:29:47.0'],
 [u'4',
  u'2008-11-28 00:08:09.0',
  u'\\N',
  u'Leila',
  u'Spencer',
  u'1447 Ross Street',
  u'San Mateo',
  u'CA',
  u'94444',
  u'6503198619',
  u'2014-03-18 13:29:47.0',
  u'2014-03-18 13:29:47.0'],
 [u'5',
  u'2008-11-15 23:06:06.0',
  u'\\N',
  u'Anita',
  u'Laughlin',
  u'2767 Hill Street',
  u'Richmond',
  u'CA',
  u'94872',
  u'5107754354',
  u'2014-03-18 13:

In [7]:
basestations.take(5)

[[u'1', u'86502', u'Chambers', u'AZ', u'35.2375', u'-109.523'],
 [u'2', u'86514', u'Teec Nos Pos', u'AZ', u'36.7797', u'-109.359'],
 [u'3', u'85602', u'Benson', u'AZ', u'31.9883', u'-110.2941'],
 [u'4', u'86011', u'Flagstaff', u'AZ', u'35.6308', u'-112.0524'],
 [u'5', u'86016', u'Gray Mountain', u'AZ', u'35.6308', u'-112.0524']]

In [8]:
#How many users are there in total?
accounts.count()

129764

In [9]:
#Take cities from accounts rds
cities = accounts.map(lambda fields: (fields[6]))

In [10]:
#Display 5 cities
cities.take(5)

[u'Oakland', u'San Francisco', u'San Mateo', u'San Mateo', u'Richmond']

In [11]:
#Get unique cities
cities_unique = cities.distinct()

In [12]:
#How many cities have users
cities_unique.count()

56

In [13]:
#Get unique cities per basestations
cities_basestations = basestations.map(lambda fields: fields[2]).distinct()

In [14]:
#How many citeis have base stations
cities_basestations.count()

238

In [15]:
#How many basestation are there in total?
basestations.count()

377

# More detailed Analysis

In [16]:
#Get all accounts in the Oregon state
oregon = accounts.filter(lambda x: "OR" in x[7])

In [17]:
oregon.count()

13575

In [18]:
oregon.first()

[u'3509',
 u'2010-09-05 22:43:35.0',
 u'\\N',
 u'Cody',
 u'Martinez',
 u'265 Langtown Road',
 u'Eugene',
 u'OR',
 u'97487',
 u'5414866653',
 u'2014-03-18 13:29:53.0',
 u'2014-03-18 13:29:53.0']

In [19]:
#Get the total ammount of users per city in oregon
oregonUsers = oregon.map(lambda fields: (fields[6])).map(lambda word: (word,1))\
.reduceByKey(lambda v1,v2:	v1+v2)

In [20]:
oregonUsers.count()

7

In [21]:
#Display all the cities in oregon and their user count
for items in oregonUsers.take(7):
    print items[0], items[1]

Salem 1496
Portland 4602
Klamath Falls 1463
Medford 1511
Eugene 1520
Pendleton 1455
Bend 1528


In [22]:
#Get RDD of all users and their cities
allUsersCities = accounts.map(lambda fields: (fields[6])).map(lambda word: (word,1))\
.reduceByKey(lambda v1,v2:	v1+v2)

In [23]:
#Show the first 5  rows
allUsersCities.take(5)

[(u'Sacramento', 6820),
 (u'San Diego', 4618),
 (u'Santa Barbara', 3067),
 (u'Anaheim', 1544),
 (u'Salem', 1496)]

In [24]:
#Count how many bases are in each city
allBasesCities = basestations.map(lambda fields: (fields[2])).map(lambda word: (word,1))\
.reduceByKey(lambda v1,v2:	v1+v2)

In [25]:

allBasesCities.take(10)

[(u'Blue', 1),
 (u'Loomis', 1),
 (u'Baker City', 1),
 (u'Packwood', 1),
 (u'Bullhead City', 1),
 (u'Santa Fe Springs', 1),
 (u'Round Mountain', 1),
 (u'Apple Valley', 1),
 (u'Eldridge', 1),
 (u'Santa Rosa', 1)]

In [26]:
#Join all bases and users counts together
cityUsersBases = allUsersCities.join(allBasesCities)

In [27]:
cityUsersBases.take(10)

[(u'Carson City', (1360, 1)),
 (u'Glendale', (1493, 2)),
 (u'Richmond', (1746, 1)),
 (u'San Francisco', (3354, 15)),
 (u'Sacramento', (6820, 4)),
 (u'Flagstaff', (1181, 1)),
 (u'Long Beach', (4500, 3)),
 (u'Palm Springs', (1482, 1)),
 (u'Redding', (1587, 1)),
 (u'Riverside', (1477, 1))]

In [28]:
#Flat map RDD so it can be run in a loop
cityUsersBases = cityUsersBases.map(lambda (city, (users,bases)) : (city,users, bases))

In [29]:
cityUsersBases.take(5)

[(u'Carson City', 1360, 1),
 (u'Glendale', 1493, 2),
 (u'Richmond', 1746, 1),
 (u'San Francisco', 3354, 15),
 (u'Sacramento', 6820, 4)]

In [30]:
#Display cities with user and base counts
for items in cityUsersBases.take(32):
    print items[0], ' \t    \t Users \t' , items[1], ' Bases \t', items[2]

Carson City  	    	 Users 	1360  Bases 	1
Glendale  	    	 Users 	1493  Bases 	2
Richmond  	    	 Users 	1746  Bases 	1
San Francisco  	    	 Users 	3354  Bases 	15
Sacramento  	    	 Users 	6820  Bases 	4
Flagstaff  	    	 Users 	1181  Bases 	1
Long Beach  	    	 Users 	4500  Bases 	3
Palm Springs  	    	 Users 	1482  Bases 	1
Redding  	    	 Users 	1587  Bases 	1
Riverside  	    	 Users 	1477  Bases 	1
San Bernardino  	    	 Users 	2985  Bases 	1
Tucson  	    	 Users 	2308  Bases 	4
Anaheim  	    	 Users 	1544  Bases 	1
Portland  	    	 Users 	4602  Bases 	7
Las Vegas  	    	 Users 	4142  Bases 	7
Pasadena  	    	 Users 	2961  Bases 	2
San Diego  	    	 Users 	4618  Bases 	18
Oakland  	    	 Users 	3491  Bases 	2
San Jose  	    	 Users 	3269  Bases 	15
Alhambra  	    	 Users 	1487  Bases 	1
San Mateo  	    	 Users 	1768  Bases 	2
Stockton  	    	 Users 	3309  Bases 	2
Santa Barbara  	    	 Users 	3067  Bases 	2
Reno  	    	 Users 	2787  Bases 	2
Santa Rosa  	    	 Users 	3340  Bases 

In [31]:
#Get weblog flume data
kbreqs = sc.textFile(flumeFiles).map(lambda line: line.split(' '))


In [32]:
kbreqs.first()

[u'3.94.78.5',
 u'-',
 u'69827',
 u'[15/Sep/2013:23:58:36',
 u'+0100]',
 u'"GET',
 u'/KBDOC-00033.html',
 u'HTTP/1.0"',
 u'200',
 u'14417',
 u'"http://www.loudacre.com"',
 u'',
 u'"Loudacre',
 u'Mobile',
 u'Browser',
 u'iFruit',
 u'1"']

In [33]:
#Count how many requests each user has made
userRequests = kbreqs.map(lambda fields: (fields[2])).map(lambda word: (word,1))\
.reduceByKey(lambda v1,v2:	v1+v2)

In [34]:
userRequests.take(10)

[(u'127716', 8),
 (u'59557', 8),
 (u'38059', 2),
 (u'99252', 2),
 (u'108342', 4),
 (u'43270', 8),
 (u'87287', 10),
 (u'84224', 20),
 (u'63499', 4),
 (u'101407', 67)]

In [35]:
#Get all users and their states
userStates = accounts.map(lambda fields: (fields[0] ,fields[7]))

In [36]:
userStates.take(5)

[(u'1', u'CA'), (u'2', u'CA'), (u'3', u'CA'), (u'4', u'CA'), (u'5', u'CA')]

In [37]:
#Join user requests and states
userStateRequests = userStates.join(userRequests)\
.map(lambda (userID, (state,requests)) : (state, (requests, userID)))

In [38]:
userStateRequests.take(5)

[(u'CA', (6, u'18886')),
 (u'CA', (100, u'60986')),
 (u'CA', (10, u'41969')),
 (u'CA', (2, u'28858')),
 (u'OR', (46, u'25965'))]

In [39]:
#Find the maximum number requests per state
maxUSRs = userStateRequests.reduceByKey(max)

In [40]:
maxUSRs.take(10)

[(u'NV', (125, u'20382')),
 (u'OR', (122, u'60223')),
 (u'AZ', (120, u'50367')),
 (u'CA', (1538, u'203'))]

In [41]:
#Reorder and flat map RDD so it can be looped
maxRequests = maxUSRs.map(lambda (state, (requests,userID)) : (state,userID, requests))

In [42]:
maxRequests.count()

4

In [43]:
#Display states and the user with biggest ammount of requests
for items in maxRequests.take(4):
    print items[0], '|  Biggest user: \t' , items[1], '\t Requests: \t', items[2]

NV |  Biggest user: 	20382 	 Requests: 	125
OR |  Biggest user: 	60223 	 Requests: 	122
AZ |  Biggest user: 	50367 	 Requests: 	120
CA |  Biggest user: 	203 	 Requests: 	1538
